# Inicialização

## Carregando bibliotecas e funções de auxílio

In [1]:
from sklearn.base import BaseEstimator
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import xml.etree.ElementTree as et


def parse_xml_to_df(xml_root):
    # Cria um dataframe vazio em que as linhas serão concatenadas.
    df = pd.DataFrame(columns=['similarity', 't', 'h'])
    # Para cada par na root.
    for pair in xml_root:
        # Recupera o valor de t.
        t = pair[0].text
        # Recupera o valor de h.
        h = pair[1].text
        # Recupera o valor da variável target.
        is_entailment = pair.attrib['entailment'] == 'Entailment'
        # Recupera o valor de similaridade atribuído.
        similarity = float(pair.attrib['similarity'])
        # Constroi a nova linha.
        new_line = pd.DataFrame([{'t': t, 'h': h, 'similarity': similarity, 'is_entailment': is_entailment}])
        # Adiciona ao dataframe.
        df = pd.concat([df, new_line], ignore_index=True)
    return df

    
class GenericEstimator(BaseEstimator):
    # Classificador genérico para usar o gridsearch com múltiplos algoritmos.
    def fit(self): pass
    
    def score(self): pass


def get_top_n_importances(data, estimator, n=10):
    # Recupera a importância do estimador.
    importances = estimator.feature_importances_
    # Recupera o nome das colunas.
    column_names = X_train.columns
    # Cria um dataframe com as importâncias.
    df_importances = pd.DataFrame({'column': column_names, 'importance': importances})
    # Recupera os top n importances.
    return df_importances.nlargest(n=n, columns='importance', keep='first')


def get_classification_results(y_train, y_train_pred, y_dev, y_dev_pred, y_test, y_test_pred):
    # Cria um dataframe com todos o resultado de todas as métricas.
    return pd.DataFrame({
        'type':['train', 'dev', 'test'],
        
        'f1': [f1_score(y_train, y_train_pred), 
               f1_score(y_dev, y_dev_pred), 
               f1_score(y_test, y_test_pred)],

        'precision': [precision_score(y_train, y_train_pred), 
                      precision_score(y_dev, y_dev_pred), 
                      precision_score(y_test, y_test_pred)],

        'recall': [recall_score(y_train, y_train_pred), 
                      recall_score(y_dev, y_dev_pred), 
                      recall_score(y_test, y_test_pred)],

        'accuracy': [accuracy_score(y_train, y_train_pred), 
                     accuracy_score(y_dev, y_dev_pred), 
                     accuracy_score(y_test, y_test_pred)],
    })

## Carregando conjuntos de dados

In [2]:
# Recupera o arquivo de entrada de treinamento.
train_xml_root = et.parse('../data/assin2-train.xml').getroot()
# Cria o dataframe de treinamento.
df_train = parse_xml_to_df(train_xml_root)

# Recupera o arquivo de entrada de validação.
dev_xml_root = et.parse('../data/assin2-dev.xml').getroot()
# Cria o dataframe de validação.
df_dev = parse_xml_to_df(dev_xml_root)

# Recupera o arquivo de entrada de validação.
test_xml_root = et.parse('../data/assin2-test.xml').getroot()
df_test = parse_xml_to_df(test_xml_root)

In [3]:
print('Shape de treinamento:', df_train.shape)
print('Shape de validação:', df_dev.shape)
print('Shape de teste:', df_test.shape)

Shape de treinamento: (6500, 4)
Shape de validação: (500, 4)
Shape de teste: (2448, 4)


## Transformação dos dados

In [4]:
# Criando uma coluna que concatena as duas frases em uma nas bases de dados.
df_train['t_h'] = df_train['t'] + ' ' + df_train['h']  
df_dev['t_h'] = df_dev['t'] + ' ' + df_dev['h']  
df_test['t_h'] = df_test['t'] + ' ' + df_test['h']  

# Modelagem

## Abordagem 1: BoW concatenando ambas as frases em uma única frase

### Preparação dos dados

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Instancia um objeto para realizar o bag of words.
vectorizer = CountVectorizer()
# Ajusta o BoW no conjunto de treinamento.
vectorizer.fit(df_train['t_h'])

# Vetorizando o conjunto de treinamento.
df_train_vec = pd.DataFrame(vectorizer.transform(df_train['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_train_vec['is_entailment'] = df_train['is_entailment']

# Vetorizando o conjunto de validação.
df_dev_vec = pd.DataFrame(vectorizer.transform(df_dev['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_dev_vec['is_entailment'] = df_dev['is_entailment']

# Vetorizando o conjunto de validação.
df_test_vec = pd.DataFrame(vectorizer.transform(df_test['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_test_vec['is_entailment'] = df_test['is_entailment']

X_train, y_train = df_train_vec.drop(columns='is_entailment'), df_train_vec['is_entailment'].astype(int)
X_dev, y_dev = df_dev_vec.drop(columns='is_entailment'), df_dev_vec['is_entailment'].astype(int)
X_test, y_test = df_test_vec.drop(columns='is_entailment'), df_test_vec['is_entailment'].astype(int)

print('Shape de treinamento:', X_train.shape, y_train.shape)
print('Shape de validação:', X_dev.shape, y_dev.shape)
print('Shape de teste:', X_test.shape, y_test.shape)

Shape de treinamento: (6500, 2310) (6500,)
Shape de validação: (500, 2310) (500,)
Shape de teste: (2448, 2310) (2448,)


### Treinamento do modelo

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import xgboost as xgb

# Cria o pipeline de transformação.
pipe = Pipeline([('clf', GenericEstimator())])

# Define um espaço de busca com diferentes algoritmos.
search_space = [
    {'clf': [LogisticRegression()],
     'clf__penalty': ['l1', 'l2'],
    },
    
    {'clf': [xgb.XGBClassifier()],
     'clf__n_estimators': [150],
    'clf__max_depth': [2, 10]},
    
    {'clf': [RandomForestClassifier()],
     'clf__n_estimators': [150, 250],
     'clf__max_depth': [2, 10, 20]},

]
    
# Cria um objeto de busca em grid com semente setada para reprodutibilidade.    
grid = GridSearchCV(
    pipe,
    search_space, 
    n_jobs=5,
    cv=5,
    scoring='f1',
    verbose=1
)

# Ajusta no conjunto de treinamento.
grid.fit(X_train, y_train)
# Recupera os resultados da validação cruzada.
df_first_training_results = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score', 'std_test_score']]

# Recupera os resultados
y_train_pred = grid.predict(X_train)
y_dev_pred = grid.predict(X_dev)
y_test_pred = grid.predict(X_test)

# Recuperando as importâncias das features.
df_first_approach_importances = get_top_n_importances(X_train, grid.best_estimator_[0])
display(first_approach_importances)

# Calcula as métricas de performance dos resultados do modelo.
df_first_approach_results = get_classification_results(
    y_train, 
    y_train_pred, 
    y_dev, 
    y_dev_pred, 
    y_test, 
    y_test_pred)
display(df_first_approach_results)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


,column,importance
1525,não,0.102028
2154,tofu,0.010888
1086,fundo,0.010294
685,dele,0.008303
1766,por,0.007710
1475,mãos,0.006723
1512,ninguém,0.006547
2016,senhora,0.006521
2307,óculos,0.006303
1146,guardando,0.006234


,type,f1,precision,recall,accuracy
0,train,0.939864,0.921123,0.959385,0.938615
1,dev,0.884921,0.877953,0.892000,0.884000
2,test,0.772872,0.680772,0.893791,0.737337


## Abordagem 2: BoW concatenando ambas as frases em uma única frase com PCA

### Treinamento do modelo

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import xgboost as xgb


# Cria o pipeline de transformação.
pipe = Pipeline([
    ('pca', PCA(random_state=42)),
    ('clf', GenericEstimator())
])

# Define um espaço de busca com diferentes algoritmos.
search_space = [
    {'clf': [LogisticRegression()],
     'clf__penalty': ['l1', 'l2'],
     'pca': [PCA(random_state=42)],
     'pca__n_components': [2, 3, 4, 5, 10, 20, 100]
    },
    
    {'clf': [xgb.XGBClassifier()],
     'clf__n_estimators': [150],
     'clf__max_depth': [2, 10],
     'pca': [PCA(random_state=42)],
     'pca__n_components': [2, 3, 4, 5, 10, 20, 100]
    },
    
    {'clf': [RandomForestClassifier()],
     'clf__n_estimators': [150, 250],
     'clf__max_depth': [2, 10, 20],
     'pca': [PCA(random_state=42)],
     'pca__n_components': [2, 3, 4, 5, 10, 20, 100]
    },
]
    
# Cria um objeto de busca em grid com semente setada para reprodutibilidade.    
grid = GridSearchCV(
    pipe,
    search_space, 
    n_jobs=5,
    cv=5,
    scoring='f1',
    verbose=1
)

# Ajusta no conjunto de treinamento.
grid.fit(X_train, y_train)
# Recupera os resultados da validação cruzada.
df_second_training_results = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score', 'std_test_score']]
# Recupera os resultados
y_train_pred = grid.predict(X_train)
y_dev_pred = grid.predict(X_dev)
y_test_pred = grid.predict(X_test)

# Calcula as métricas de performance dos resultados do modelo.
df_second_approach_results = get_classification_results(
    y_train, 
    y_train_pred, 
    y_dev, 
    y_dev_pred, 
    y_test, 
    y_test_pred)
display(df_second_approach_results)

Fitting 5 folds for each of 70 candidates, totalling 350 fits


/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
35 fits failed out of a total of 350.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/linear_model/_l

,type,f1,precision,recall,accuracy
0,train,0.949439,0.948419,0.950462,0.949385
1,dev,0.877551,0.895833,0.860000,0.880000
2,test,0.700511,0.600818,0.839869,0.640931


# Abordagem 3: TF-IDF concatenando ambas as frases em uma única frase

### Preparação dos dados

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instancia um objeto para realizar o bag of words.
vectorizer = TfidfVectorizer()
# Ajusta o BoW no conjunto de treinamento.
vectorizer.fit(df_train['t_h'])

# Vetorizando o conjunto de treinamento.
df_train_vec = pd.DataFrame(vectorizer.transform(df_train['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_train_vec['is_entailment'] = df_train['is_entailment']

# Vetorizando o conjunto de validação.
df_dev_vec = pd.DataFrame(vectorizer.transform(df_dev['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_dev_vec['is_entailment'] = df_dev['is_entailment']

# Vetorizando o conjunto de validação.
df_test_vec = pd.DataFrame(vectorizer.transform(df_test['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_test_vec['is_entailment'] = df_test['is_entailment']

X_train, y_train = df_train_vec.drop(columns='is_entailment'), df_train_vec['is_entailment'].astype(int)
X_dev, y_dev = df_dev_vec.drop(columns='is_entailment'), df_dev_vec['is_entailment'].astype(int)
X_test, y_test = df_test_vec.drop(columns='is_entailment'), df_test_vec['is_entailment'].astype(int)

print('Shape de treinamento:', X_train.shape, y_train.shape)
print('Shape de validação:', X_dev.shape, y_dev.shape)
print('Shape de teste:', X_test.shape, y_test.shape)

Shape de treinamento: (6500, 2310) (6500,)
Shape de validação: (500, 2310) (500,)
Shape de teste: (2448, 2310) (2448,)


### Treinamento do modelo

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import xgboost as xgb

# Cria o pipeline de transformação.
pipe = Pipeline([('clf', GenericEstimator())])

# Define um espaço de busca com diferentes algoritmos.
search_space = [
    {'clf': [LogisticRegression()],
     'clf__penalty': ['l1', 'l2'],
    },
    
    {'clf': [xgb.XGBClassifier()],
     'clf__n_estimators': [150],
    'clf__max_depth': [2, 10]},
    
    {'clf': [RandomForestClassifier()],
     'clf__n_estimators': [150, 250],
     'clf__max_depth': [2, 10, 20]},

]
    
# Cria um objeto de busca em grid com semente setada para reprodutibilidade.    
grid = GridSearchCV(
    pipe,
    search_space, 
    n_jobs=5,
    cv=5,
    scoring='f1',
    verbose=1
)

# Ajusta no conjunto de treinamento.
grid.fit(X_train, y_train)
# Recupera os resultados da validação cruzada.
df_third_training_results = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score', 'std_test_score']]

# Recupera os resultados
y_train_pred = grid.predict(X_train)
y_dev_pred = grid.predict(X_dev)
y_test_pred = grid.predict(X_test)

# Recuperando as importâncias das features.
df_third_approach_importances = get_top_n_importances(X_train, grid.best_estimator_[0])
display(df_third_approach_importances)

# Calcula as métricas de performance dos resultados do modelo.
df_third_approach_results = get_classification_results(
    y_train, 
    y_train_pred, 
    y_dev, 
    y_dev_pred, 
    y_test, 
    y_test_pred)
display(df_third_approach_results)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/linear_model/_logi

,column,importance
1525,não,0.058896
592,corajosamente,0.011095
1086,fundo,0.010216
1621,passeando,0.009011
548,comidas,0.008843
1229,jovens,0.008487
406,capa,0.008455
1780,pouco,0.008015
1512,ninguém,0.007976
1571,ovo,0.007768


,type,f1,precision,recall,accuracy
0,train,0.949153,0.933889,0.964923,0.948308
1,dev,0.890688,0.901639,0.880000,0.892000
2,test,0.748086,0.651515,0.878268,0.704248


## Abordagem 4: BoW concatenando ambas as frases em uma única frase e n-grams=(1,3)

### Preparação dos dados

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Instancia um objeto para realizar o bag of words com unigramas até trigramas.
vectorizer = CountVectorizer(ngram_range=(1,3))
# Ajusta o BoW no conjunto de treinamento.
vectorizer.fit(df_train['t_h'])

# Vetorizando o conjunto de treinamento.
df_train_vec = pd.DataFrame(vectorizer.transform(df_train['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_train_vec['is_entailment'] = df_train['is_entailment']

# Vetorizando o conjunto de validação.
df_dev_vec = pd.DataFrame(vectorizer.transform(df_dev['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_dev_vec['is_entailment'] = df_dev['is_entailment']

# Vetorizando o conjunto de validação.
df_test_vec = pd.DataFrame(vectorizer.transform(df_test['t_h']).toarray(), columns=vectorizer.get_feature_names_out())
df_test_vec['is_entailment'] = df_test['is_entailment']

X_train, y_train = df_train_vec.drop(columns='is_entailment'), df_train_vec['is_entailment'].astype(int)
X_dev, y_dev = df_dev_vec.drop(columns='is_entailment'), df_dev_vec['is_entailment'].astype(int)
X_test, y_test = df_test_vec.drop(columns='is_entailment'), df_test_vec['is_entailment'].astype(int)

print('Shape de treinamento:', X_train.shape, y_train.shape)
print('Shape de validação:', X_dev.shape, y_dev.shape)
print('Shape de teste:', X_test.shape, y_test.shape)

Shape de treinamento: (6500, 33527) (6500,)
Shape de validação: (500, 33527) (500,)
Shape de teste: (2448, 33527) (2448,)


### Treinamento do modelo

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import xgboost as xgb

# Cria o pipeline de transformação.
pipe = Pipeline([('clf', GenericEstimator())])

# Define um espaço de busca com diferentes algoritmos.
search_space = [
    {'clf': [LogisticRegression()],
     'clf__penalty': ['l1', 'l2'],
    },
    
    {'clf': [xgb.XGBClassifier()],
     'clf__n_estimators': [150],
    'clf__max_depth': [2, 10]},
    
    {'clf': [RandomForestClassifier()],
     'clf__n_estimators': [150, 250],
     'clf__max_depth': [2, 10, 20]},

]
    
# Cria um objeto de busca em grid com semente setada para reprodutibilidade.    
grid = GridSearchCV(
    pipe,
    search_space, 
    n_jobs=5,
    cv=5,
    scoring='f1',
    verbose=1
)

# Ajusta no conjunto de treinamento.
grid.fit(X_train, y_train)
# Recupera os resultados da validação cruzada.
df_fourth_training_results = pd.DataFrame(grid.cv_results_)[['params', 'mean_test_score', 'std_test_score']]

# Recupera os resultados
y_train_pred = grid.predict(X_train)
y_dev_pred = grid.predict(X_dev)
y_test_pred = grid.predict(X_test)

# Recuperando as importâncias das features.
df_fourth_approach_importances = get_top_n_importances(X_train, grid.best_estimator_[0])
display(df_fourth_approach_importances)

# Calcula as métricas de performance dos resultados do modelo.
df_fourth_approach_results = get_classification_results(
    y_train, 
    y_train_pred, 
    y_dev, 
    y_dev_pred, 
    y_test, 
    y_test_pred)
display(df_fourth_approach_results)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/vfsilva/virtualenvs/phd-env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

# Comparação entre abordagens